This notebook is for performing tests on the permutation algorithm that calculates $\tilde{\mathbf{B}}^{-1}\mathbf{a}$.

In [2]:
import numpy as np
def test(mat, ds, K, idxs):
    print("Initial matrix")
    print(mat)

    print("\nNow, swap relevant columns")
    for i, val in enumerate(idxs):
      offset = np.sum(ds[:i])
      mat[:, offset:val+1+offset] = np.roll(mat[:, offset:val+1+offset], 1, axis=1)
    print(mat)

    print("\nAnd the row chunks")
    for i, val in enumerate(idxs):
      chunk_size = np.prod(ds[:i])
      num_chunks = np.prod(ds[i+1:])

      # Break up into chunk_size blocks
      last_row = mat[-1:]
      mat = mat[:-1]
      split_mat = np.array(np.split(mat, num_chunks))
      split_mat[:, :(val+1)*chunk_size] = np.roll(
          split_mat[:, :(val+1)*chunk_size],
          chunk_size,
          axis=1
      )
      mat = np.concatenate([
        split_mat.reshape(-1, split_mat.shape[-1]),
        last_row
      ])
    print(mat)

    print("Now grab the rows we want")
    shrunk = mat[0:1] # First row
    mat = mat[1:]
    last_row = mat[-1:]
    for i, val in enumerate(ds):
      step_size = np.prod(ds[:i])
      amount = val-1
      shrunk = np.concatenate([
        shrunk,
        mat[0::step_size][:amount]
      ])
      mat = mat[step_size*amount:]
    shrunk = np.concatenate([
      shrunk,
      last_row
    ])
    print(shrunk)

    print("Move columns to end")
    last_col = shrunk[:, -1:]
    shrunk = shrunk[:, :-1]
    for i, val in enumerate(idxs):
      # We subtract `i` to account for the fact that we've
      # already moved earlier columns!
      offset = np.sum(ds[:i])-i
      shrunk[:, offset:] = np.roll(shrunk[:, offset:], -1, axis=1)
    shrunk = np.concatenate([
      shrunk,
      last_col
    ], axis=1)
    print(shrunk)

    print("Move top row down, add identity")
    last_row = shrunk[-1:]
    shrunk = shrunk[:-1]
    shrunk = np.roll(shrunk, -1, axis=0)
    shrunk = np.concatenate([
      shrunk,
      np.concatenate([
        np.zeros((K-1, np.sum(ds)-K+1)),
        np.concatenate([
          np.eye(K-1),
          300 + np.arange(K-1).reshape(K-1, 1)
        ], axis=1)
      ], axis=1),
      last_row
    ])
    print(shrunk)

    print("Invert B")
    B_INVERSE = np.linalg.inv(shrunk[:-1, :-1])
    shrunk[:-1, :-1] = B_INVERSE.T
    shrunk = shrunk.T
    print(shrunk)

    print("Output indices")
    print(shrunk[-1, :sum(ds-1)+1] - 101)


    

In [6]:
ds = np.array([4, 3])
K = len(ds)
idxs = [2, 2]

mat = np.array([
  [1, 0, 0, 0, 1, 0, 0, 101],
  [0, 1, 0, 0, 1, 0, 0, 102],
  [0, 0, 1, 0, 1, 0, 0, 103],
  [0, 0, 0, 1, 1, 0, 0, 104],
  [1, 0, 0, 0, 0, 1, 0, 105],
  [0, 1, 0, 0, 0, 1, 0, 106],
  [0, 0, 1, 0, 0, 1, 0, 107],
  [0, 0, 0, 1, 0, 1, 0, 108],
  [1, 0, 0, 0, 0, 0, 1, 109],
  [0, 1, 0, 0, 0, 0, 1, 110],
  [0, 0, 1, 0, 0, 0, 1, 111],
  [0, 0, 0, 1, 0, 0, 1, 112],
  [2, 3, 4, 5, 6, 7, 8, 200]
])
test(mat, ds, K, idxs)

Initial matrix
[[  1   0   0   0   1   0   0 101]
 [  0   1   0   0   1   0   0 102]
 [  0   0   1   0   1   0   0 103]
 [  0   0   0   1   1   0   0 104]
 [  1   0   0   0   0   1   0 105]
 [  0   1   0   0   0   1   0 106]
 [  0   0   1   0   0   1   0 107]
 [  0   0   0   1   0   1   0 108]
 [  1   0   0   0   0   0   1 109]
 [  0   1   0   0   0   0   1 110]
 [  0   0   1   0   0   0   1 111]
 [  0   0   0   1   0   0   1 112]
 [  2   3   4   5   6   7   8 200]]

Now, swap relevant columns
[[  0   1   0   0   0   1   0 101]
 [  0   0   1   0   0   1   0 102]
 [  1   0   0   0   0   1   0 103]
 [  0   0   0   1   0   1   0 104]
 [  0   1   0   0   0   0   1 105]
 [  0   0   1   0   0   0   1 106]
 [  1   0   0   0   0   0   1 107]
 [  0   0   0   1   0   0   1 108]
 [  0   1   0   0   1   0   0 109]
 [  0   0   1   0   1   0   0 110]
 [  1   0   0   0   1   0   0 111]
 [  0   0   0   1   1   0   0 112]
 [  4   2   3   5   8   6   7 200]]

And the row chunks
[[  1   0   0   0   1   0

In [55]:
ds = np.array([2, 2, 3, 2])
K = len(ds)
idxs = [1, 0, 1, 1]

mat = np.array([
  [1, 0, 1, 0, 1, 0, 0, 1, 0, 101],
  [0, 1, 1, 0, 1, 0, 0, 1, 0, 102],
  [1, 0, 0, 1, 1, 0, 0, 1, 0, 103],
  [0, 1, 0, 1, 1, 0, 0, 1, 0, 104],
  [1, 0, 1, 0, 0, 1, 0, 1, 0, 105],
  [0, 1, 1, 0, 0, 1, 0, 1, 0, 106],
  [1, 0, 0, 1, 0, 1, 0, 1, 0, 107],
  [0, 1, 0, 1, 0, 1, 0, 1, 0, 108],
  [1, 0, 1, 0, 0, 0, 1, 1, 0, 109],
  [0, 1, 1, 0, 0, 0, 1, 1, 0, 110],
  [1, 0, 0, 1, 0, 0, 1, 1, 0, 111],
  [0, 1, 0, 1, 0, 0, 1, 1, 0, 112],
  [1, 0, 1, 0, 1, 0, 0, 0, 1, 113],
  [0, 1, 1, 0, 1, 0, 0, 0, 1, 114],
  [1, 0, 0, 1, 1, 0, 0, 0, 1, 115],
  [0, 1, 0, 1, 1, 0, 0, 0, 1, 116],
  [1, 0, 1, 0, 0, 1, 0, 0, 1, 117],
  [0, 1, 1, 0, 0, 1, 0, 0, 1, 118],
  [1, 0, 0, 1, 0, 1, 0, 0, 1, 119],
  [0, 1, 0, 1, 0, 1, 0, 0, 1, 120],
  [1, 0, 1, 0, 0, 0, 1, 0, 1, 121],
  [0, 1, 1, 0, 0, 0, 1, 0, 1, 122],
  [1, 0, 0, 1, 0, 0, 1, 0, 1, 123],
  [0, 1, 0, 1, 0, 0, 1, 0, 1, 124],
  [2, 3, 4, 5, 6, 7, 8, 9, -1, 200]
])
test(mat, ds, K, idxs)

Initial matrix
[[  1   0   1   0   1   0   0   1   0 101]
 [  0   1   1   0   1   0   0   1   0 102]
 [  1   0   0   1   1   0   0   1   0 103]
 [  0   1   0   1   1   0   0   1   0 104]
 [  1   0   1   0   0   1   0   1   0 105]
 [  0   1   1   0   0   1   0   1   0 106]
 [  1   0   0   1   0   1   0   1   0 107]
 [  0   1   0   1   0   1   0   1   0 108]
 [  1   0   1   0   0   0   1   1   0 109]
 [  0   1   1   0   0   0   1   1   0 110]
 [  1   0   0   1   0   0   1   1   0 111]
 [  0   1   0   1   0   0   1   1   0 112]
 [  1   0   1   0   1   0   0   0   1 113]
 [  0   1   1   0   1   0   0   0   1 114]
 [  1   0   0   1   1   0   0   0   1 115]
 [  0   1   0   1   1   0   0   0   1 116]
 [  1   0   1   0   0   1   0   0   1 117]
 [  0   1   1   0   0   1   0   0   1 118]
 [  1   0   0   1   0   1   0   0   1 119]
 [  0   1   0   1   0   1   0   0   1 120]
 [  1   0   1   0   0   0   1   0   1 121]
 [  0   1   1   0   0   0   1   0   1 122]
 [  1   0   0   1   0   0   1   0   1 1

# Better way to shuffle

In [124]:
def test_2(mat, ds, K, idxs):
    rows = []
    chunk_size = np.array([np.prod(ds[:ell]) for ell in range(K)])
    csidx = chunk_size @ idxs
    for ell, d in enumerate(ds):
        start = csidx - idxs[ell]*np.prod(ds[:ell])
        rows += [idx for k in range(d) if (idx:=start + k*chunk_size[ell]) != csidx]
    rows += [csidx]
    return mat[rows]

def test_2_v2(mat, ds, K, idxs):
    rows = [] * (np.sum(ds) - K + 1)
    chunk_size = np.array([np.prod(ds[:ell]) for ell in range(K)])
    csidx = chunk_size @ idxs
    x = 0
    for ell, d in enumerate(ds):
        start = csidx - idxs[ell]*np.prod(ds[:ell])
        rows[x:x+d-1] = np.array([
            idx for k in range(d) if (idx:=start + k*chunk_size[ell]) != csidx
        ])
        x+=d-1
    rows += [csidx]
    return mat[rows]

def test_2_v3(mat, ds, K, idxs):
    rows = []
    chunk_size = np.array([np.prod(ds[:ell]) for ell in range(K)])
    csidx = chunk_size @ idxs
    for ell, d in enumerate(ds):
        start = csidx - idxs[ell]*np.prod(ds[:ell])
        rows += [
            start + np.delete(
                np.arange(d),
                (csidx - start) // chunk_size[ell]
            )* chunk_size[ell]
        ]
    rows += [np.array([csidx])]
    return mat[list(np.concatenate(rows))]

def test_2_v4(mat, ds, K, idxs):
    rows = []
    chunk_size = np.array([np.prod(ds[:ell]) for ell in range(K)])
    csidx = chunk_size @ idxs
    for ell, d in enumerate(ds):
        start = csidx - idxs[ell]*np.prod(ds[:ell])
        crossover = (csidx - start) // chunk_size[ell]
        rows += [
            start + np.arange(crossover) * chunk_size[ell]
        ] + [
            start + np.arange(crossover+1, d) * chunk_size[ell]
        ]
    rows += [np.array([csidx])]
    return mat[list(np.concatenate(rows))]

In [125]:
ds = np.array([2, 2, 3, 2])
K = len(ds)
idxs = [1, 0, 1, 1]

mat = np.array([
  [1, 0, 1, 0, 1, 0, 0, 1, 0, 101],
  [0, 1, 1, 0, 1, 0, 0, 1, 0, 102],
  [1, 0, 0, 1, 1, 0, 0, 1, 0, 103],
  [0, 1, 0, 1, 1, 0, 0, 1, 0, 104],
  [1, 0, 1, 0, 0, 1, 0, 1, 0, 105],
  [0, 1, 1, 0, 0, 1, 0, 1, 0, 106],
  [1, 0, 0, 1, 0, 1, 0, 1, 0, 107],
  [0, 1, 0, 1, 0, 1, 0, 1, 0, 108],
  [1, 0, 1, 0, 0, 0, 1, 1, 0, 109],
  [0, 1, 1, 0, 0, 0, 1, 1, 0, 110],
  [1, 0, 0, 1, 0, 0, 1, 1, 0, 111],
  [0, 1, 0, 1, 0, 0, 1, 1, 0, 112],
  [1, 0, 1, 0, 1, 0, 0, 0, 1, 113],
  [0, 1, 1, 0, 1, 0, 0, 0, 1, 114],
  [1, 0, 0, 1, 1, 0, 0, 0, 1, 115],
  [0, 1, 0, 1, 1, 0, 0, 0, 1, 116],
  [1, 0, 1, 0, 0, 1, 0, 0, 1, 117],
  [0, 1, 1, 0, 0, 1, 0, 0, 1, 118],
  [1, 0, 0, 1, 0, 1, 0, 0, 1, 119],
  [0, 1, 0, 1, 0, 1, 0, 0, 1, 120],
  [1, 0, 1, 0, 0, 0, 1, 0, 1, 121],
  [0, 1, 1, 0, 0, 0, 1, 0, 1, 122],
  [1, 0, 0, 1, 0, 0, 1, 0, 1, 123],
  [0, 1, 0, 1, 0, 0, 1, 0, 1, 124],
  [2, 3, 4, 5, 6, 7, 8, 9, -1, 200]
])
print(test_2(mat, ds, K, idxs))
print(test_2_v2(mat, ds, K, idxs))
print(test_2_v3(mat, ds, K, idxs))
print(test_2_v4(mat, ds, K, idxs))

[[  1   0   1   0   0   1   0   0   1 117]
 [  0   1   0   1   0   1   0   0   1 120]
 [  0   1   1   0   1   0   0   0   1 114]
 [  0   1   1   0   0   0   1   0   1 122]
 [  0   1   1   0   0   1   0   1   0 106]
 [  0   1   1   0   0   1   0   0   1 118]]
[[  1   0   1   0   0   1   0   0   1 117]
 [  0   1   0   1   0   1   0   0   1 120]
 [  0   1   1   0   1   0   0   0   1 114]
 [  0   1   1   0   0   0   1   0   1 122]
 [  0   1   1   0   0   1   0   1   0 106]
 [  0   1   1   0   0   1   0   0   1 118]]
[[  1   0   1   0   0   1   0   0   1 117]
 [  0   1   0   1   0   1   0   0   1 120]
 [  0   1   1   0   1   0   0   0   1 114]
 [  0   1   1   0   0   0   1   0   1 122]
 [  0   1   1   0   0   1   0   1   0 106]
 [  0   1   1   0   0   1   0   0   1 118]]
[[  1   0   1   0   0   1   0   0   1 117]
 [  0   1   0   1   0   1   0   0   1 120]
 [  0   1   1   0   1   0   0   0   1 114]
 [  0   1   1   0   0   0   1   0   1 122]
 [  0   1   1   0   0   1   0   1   0 106]
 [  0   

# Better Way to Roll:
ignore, superseeded by above section

In [4]:
import numpy as np
%load_ext line_profiler

In [20]:
# EXPERIMENTS
def naive_roll(mat, val, num_chunks, chunk_size):
  split_mat = mat.reshape(num_chunks, -1)
  split_mat[:, :(val+1)*chunk_size] = np.roll(
      split_mat[:, :(val+1)*chunk_size],
      chunk_size,
      axis=1
  )
  mat = split_mat.reshape(-1)
  return mat
  
def smart_roll(mat, val, num_chunks, chunk_size):
  split_mat = mat.reshape(num_chunks, -1)
  temp = split_mat[:, val*chunk_size:(val+1)*chunk_size].copy()
  split_mat[:, chunk_size:(val+1)*chunk_size] = split_mat[:, :val*chunk_size]
  split_mat[:, :chunk_size] = temp
  mat = split_mat.reshape(-1)
  return mat

def together(x):
    naive_roll(rando, 1, 5, 2)
    smart_roll(rando, 1, 5, 2)
  
rando = np.arange(2000)
%lprun -f together -f smart_roll -f naive_roll together(rando)